In [1]:
import mediapipe
import cv2
import numpy as np
import requests
import threading

In [2]:
drawingModule = mediapipe.solutions.drawing_utils
handsModule = mediapipe.solutions.hands

In [3]:
def req(m):
    requests.get("http://blr1.blynk.cloud/external/api/update?token=W_AEA0TggsKjcyOuQbOMLbD1_fsNMpvs&v0=" + m)

def send(m):
    threading.Thread(target = req, args = (m), daemon = True).start()

In [4]:
cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

with handsModule.Hands(static_image_mode = False, min_detection_confidence = 0.5, min_tracking_confidence = 0.5, max_num_hands = 2) as hands:
     while True:
          ret, frame = cap.read()

          frame1 = cv2.resize(frame, (640, 480))
          image_height, image_width, _ = frame1.shape
           
          results = hands.process(cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB))
      
          if results.multi_hand_landmarks != None:
               arr = np.array(results.multi_hand_landmarks)
               no_hands = arr.shape[0]   
               for n in range(0, no_hands):
                    arrx = []; arry = []; 
                    arr = np.array(results.multi_hand_landmarks[n].landmark)
                    for i in range (1, arr.size, 4):
                         tempx = []; tempy = []
                         for j in range(i, i + 4 ):
                              tempx.append(arr[j].x)
                              tempy.append(arr[j].y)
                         arrx.append(tempx)
                         arry.append(tempy)
                    if no_hands == 1 and n == 0:
                         if arry[1][3] > arry[1][0] and arry[2][3] > arry[2][0] and arry[3][3] > arry[3][0] and arry[4][3] > arry[4][0]:
                              cv2.putText(frame1, "Stop!" ,(int(arrx[1][3]* image_width), int(arry[1][3] * image_height)- 100), cv2.FONT_HERSHEY_COMPLEX,(0.8),(0,0,255),2) 
                              m = "0"
                         elif arry[1][3] < arry[1][0] and arry[2][3] > arry[2][0] and arry[3][3] > arry[3][0] and arry[4][3] > arry[4][0]:
                              cv2.putText(frame1, "Forward!" ,(int(arrx[1][3]* image_width), int(arry[1][3] * image_height)- 100), cv2.FONT_HERSHEY_COMPLEX,(0.8),(0,0,255),2) 
                              m = "1"
                         elif arry[1][3] < arry[1][0] and arry[2][3] < arry[2][0] and arry[3][3] > arry[3][0] and arry[4][3] > arry[4][0]:
                              cv2.putText(frame1, "Backward!" ,(int(arrx[1][3]* image_width), int(arry[1][3] * image_height)- 100), cv2.FONT_HERSHEY_COMPLEX,(0.8),(0,0,255),2) 
                              m = "2"
                         elif arry[1][3] < arry[1][0] and arry[2][3] < arry[2][0] and arry[3][3] < arry[3][0] and arry[4][3] > arry[4][0]:
                              cv2.putText(frame1, "Right!" ,(int(arrx[2][3]* image_width), int(arry[2][3] * image_height)- 100), cv2.FONT_HERSHEY_COMPLEX,(0.8),(0,0,255),2)
                              m = "3"
                         elif arry[1][3] < arry[1][0] and arry[2][3] < arry[2][0] and arry[3][3] < arry[3][0] and arry[4][3] < arry[4][0]:
                              cv2.putText(frame1, "Left!" ,(int(arrx[2][3]* image_width), int(arry[2][3] * image_height)- 100), cv2.FONT_HERSHEY_COMPLEX,(0.8),(0,0,255),2)
                              m = "4"            
          else:
               m = "0"
          send(m)
    
          cv2.imshow("Frame", frame1)
          key = cv2.waitKey(1) & 0xFF
          if key == ord("q"):
              break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


-1